# Model 1. Bag of Words (BoW)

El primer paso es ajustar un modelo de Bolsa de Palabras visual (Visual BoW), un método que combina la extracción de características (features) a partir filtros con un modelo de Machine Learning tradicional (como puede ser SVM) para realizar la clasificación de imágenes.

El método de BoW se basa en una serie de etapas realizadas de manera secuencial. En primer lugar, se realiza la extracción de $n$ vectores de características de longitud definida mediante filtros (como pueden ser SIFT, SURF, ORB o KAZE). Una vez extraídos los vectores para cada una de las imágenes del conjunto de entrenamiento, se procede a aplicar una técnica de clustering no supervisado que permita reducir el espacio de características que utilizar como input para el modelo de clasificación. De este modo, se consigue reducir los $n$ vectores de características a un *vocabulario* compuesto por $m$ clusters de las mismas. Seguidamente, para cada una de las imágenes, se realiza un conteo del número de características pertenecientes a cada uno de los clusters generados, dando lugar a una *firma* única para cada una de dichas imágenes. Esta aproximación es análoga a la comparación de documentos dentro de un modelo de BoW tradicional, donde se realiza un histograma de frecuencia de las palabras que aparecen en cada uno de los documentos analizados para luego comparar ambos histogramas y determinan la proximidad léxica entre los mismos. Dichos histogramas se encuentran normalizados en cada una de sus bins (lo cual se consigue dividiendo el conteo por el número total de clusters realizados). Finalmente, se emplean el vector con los valores de conteo normalizados generados en el paso anterior como input para el modelo de clasificación, así como la etiqueta de clase correspondiente a cada una de las imágenes.

Algunos de los parámetros clave a considerar dentro del diseño de un clasificador BoW incluyen:

+ **Tamaño del vocabulario**: Un vocabulario visual demasiado pequeño resulta en una generalización demasiado alta, que no permite al algoritmo diferenciar con precisión las características de cada una de las clases. Esto resulta especialmente crítico en el caso de problemas multiclase, especialmente si hay varias clases que resultan similares entre sí, puesto que se requiere un nivel de precisión alto para poder diferenciar detalles críticos a la hora de clasificarlas. Por otra parte, un vocabulario demasiado grande implica un aumento de la posibilidad de *overfitting* del modelo, que impide la generalización a ejemplos no vistos durante el entrenamiento.

+ **Feature extractor**: El algoritmo empleado para realizar la extracción inicial de características a partir de las imágenes de entrada resulta igualmente importante, puesto que es en última instancia aquel capaz de determinar los keypoints que van a emplearse para la construcción de los clusters y posteriores variables de input del modelo. El algoritmo más popular para realizar esta tarea es `SIFT`. Sin embargo, existen otros algoritmos, tales como FAST, ORB, SURF o KAZE que presentan diferencias en su forma de calcular las características. En este proyecto comparamos la extracción de características realizada con los algoritmos SIFT, KAZE y FREAK (en este último empleando FAST para realizar la extracción de los keypoints de la imagen).

+ **Tipo de modelo de clasificación**: El modelo escogido para realizar la clasificación de las imágenes resulta del mismo modo importante, puesto que determinados modelos son más adecuados para determinadas tareas. Por ejemplo, en el caso de un problema de clasificación con una frontera lineal, aproximaciones como SVM resultanóptimas, dado que permiten una separación limpia de los grupos. Sin embargo, otros modelos como los basados en árboles resultan ineficientes en casos de frontera lineal, ya que se encuentran limitados a realizar cortes perpendiculares al espacio de entrada de las variables, siendo más adecuados para problemas en los que existen fronteras complejas y no lineales. Por ello, se propone la comparación de 2 tipos de modelo diferentes: `SVM` y `Random Forest`, ambos implementados en la librería de `cv2`.

En la realización de este apartado de la práctica se seguirá un proceso iterativo. En primer lugar se procederá a ajustar un modelo de BoW sencillo que pueda servir como referencia de rendimiento. Después, se irán implementando las mejoras anteriormente descritas para tratar de mejorar la calidad del modelo, realizando un seguimiento de los errores para ajustar el modelo de forma progresiva.

### Importar librerías

In [3]:
from importlib import reload

from typing import List
from bow import BoW
from dataset import Dataset
import image_classifier
from image_classifier import ImageClassifier, ImageClassifierRF
reload(image_classifier)

<module 'image_classifier' from '/home/carlotamoh/Dropbox/Documents/MasterBD/ML2/Labs/ml2-deep-learning/BoW/image_classifier.py'>

### Cargar los datos

En primer lugar procedemos a realizar la carga de los datos que usaremos tanto para entrenar al clasificador como para realizar la posterior evaluación de su performance. Para ello, podemos emplear el código proporcionado en el módulo `dataset.py`

In [4]:
training_set = Dataset.load('../dataset/dataset/training', '*.jpg')
validation_set = Dataset.load('../dataset/dataset/validation', '*.jpg')

# 1. Modelo SVM

La primera combinación sobre la cual vamos a iterar es una combinación de SIFT como extractor de features y SVM como clasificador, empleando KMeans como algoritmo de clustering. En este caso, emplearemos un vocabulario de tamaño 100.

El primer paso consiste en construir el vocabulario visual que emplearemos para el clasificador. Para ello, podemos utilizar el método `build_vocabulary` implementado en la clase `BoW`. En esta primera iteración emplearemos el algoritmo `KMeans` con un tamaño de vocabulario predeterminado (100), así como el algoritmo `SIFT` para realizar la extracción de características.

In [4]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=100)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [00:56<00:00, 53.03image/s]

Clustering descriptors into 100 words using K-means...


Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [5]:
# get vocabulary out
vocabulary = bow.vocabulary
vocabulary

array([[ 16.747385 ,  15.82425  ,  21.679478 , ...,  34.21678  ,
         63.584835 ,  42.801136 ],
       [102.15249  ,  33.8648   ,   7.11611  , ...,  13.129263 ,
         12.666279 ,  12.709755 ],
       [ 26.394022 ,   8.670212 ,   6.3081994, ...,   8.174572 ,
          5.491279 ,  18.945017 ],
       ...,
       [ 84.93458  ,  23.225725 ,   6.9048576, ...,   6.295441 ,
          5.6700916,  12.155427 ],
       [ 12.284902 ,  35.694164 ,  85.720566 , ...,  14.846972 ,
         16.770605 ,  21.496426 ],
       [ 14.037738 ,   6.829561 ,   6.5730786, ...,   6.338609 ,
          5.3614993,  11.628396 ]], dtype=float32)

Una vez hemos conseguido los clusters de características que constituyen nuestro vocabulario podemos entrenar al clasificadore de imágenes. En esta primera iteración emplearemos el modelo SVM con un kernel lineal, utilizando un máximo de 100 iteraciones.

In [6]:
image_classifier = ImageClassifier(bow)
image_classifier.train(training_set, iterations=100)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:48<00:00, 27.41image/s]

Training SVM...


Una vez hemos entrenado el modelo podemos analizar su performance en el conjunto de entrenamiento:

In [7]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             30.0    3.0     2.0      0.0        18.0          4.0      2.0         22.0       3.0     7.0           5.0   12.0     4.0     1.0            3.0
Coast                2.0  180.0    10.0      9.0         1.0          2.0      0.0          0.0      21.0     1.0          24.0    1.0     1.0     3.0            5.0
Forest               0.0    0.0   208.0      0.0         2.0          0.0      0.0          0.0       5.0     0.0          13.0    0.0     0.0     0.0            0.0
Highway              0.0   31.0     1.0     69.0         3.0          4.0      0.0          0.0       5.0     0.0          11.0    2.0    15.0     9.0           10.0
Industrial           3.0    2.0     2.0      3.0        98.0         18.0      0.0         11.0      12.0     4.0  

(0.5996649916247906,
 array([[ 30.,   3.,   2.,   0.,  18.,   4.,   2.,  22.,   3.,   7.,   5.,  12.,   4.,   1.,   3.],
        [  2., 180.,  10.,   9.,   1.,   2.,   0.,   0.,  21.,   1.,  24.,   1.,   1.,   3.,   5.],
        [  0.,   0., 208.,   0.,   2.,   0.,   0.,   0.,   5.,   0.,  13.,   0.,   0.,   0.,   0.],
        [  0.,  31.,   1.,  69.,   3.,   4.,   0.,   0.,   5.,   0.,  11.,   2.,  15.,   9.,  10.],
        [  3.,   2.,   2.,   3.,  98.,  18.,   0.,  11.,  12.,   4.,   4.,  22.,  14.,   5.,  13.],
        [  3.,   4.,   0.,   3.,  18., 140.,   0.,   5.,   0.,   1.,   3.,  13.,   6.,   1.,  11.],
        [  6.,   1.,   0.,   0.,  24.,  21.,  11.,  23.,   0.,  10.,   0.,   5.,   3.,   2.,   4.],
        [  5.,   1.,   1.,   0.,  23.,  13.,   1., 107.,   2.,   7.,   1.,  10.,   7.,   6.,   5.],
        [  0.,  20.,  17.,   0.,   0.,   0.,   0.,   2., 200.,   0.,  27.,   2.,   4.,   1.,   1.],
        [  9.,   0.,   0.,   0.,  15.,  13.,   1.,  23.,   0.,  45.,   0.,   2.

Procedemos a continuación a evaluar el modelo en el conjunto de test

In [8]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             13.0    7.0     1.0      2.0         7.0          6.0      2.0         29.0       4.0     9.0           4.0    7.0     2.0     2.0            5.0
Coast                2.0   64.0     1.0      5.0         0.0          1.0      0.0          0.0      12.0     0.0          13.0    0.0     1.0     1.0            0.0
Forest               0.0    0.0    85.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0           8.0    1.0     0.0     1.0            0.0
Highway              0.0   31.0     0.0     25.0         5.0         10.0      0.0          2.0       8.0     0.0           5.0    0.0     5.0     5.0            4.0
Industrial           5.0    2.0     0.0      1.0        34.0         18.0      1.0          6.0       3.0     0.0

(0.4686666666666667,
 array([[13.,  7.,  1.,  2.,  7.,  6.,  2., 29.,  4.,  9.,  4.,  7.,  2.,  2.,  5.],
        [ 2., 64.,  1.,  5.,  0.,  1.,  0.,  0., 12.,  0., 13.,  0.,  1.,  1.,  0.],
        [ 0.,  0., 85.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  8.,  1.,  0.,  1.,  0.],
        [ 0., 31.,  0., 25.,  5., 10.,  0.,  2.,  8.,  0.,  5.,  0.,  5.,  5.,  4.],
        [ 5.,  2.,  0.,  1., 34., 18.,  1.,  6.,  3.,  0.,  6.,  9.,  8.,  1.,  6.],
        [ 0.,  0.,  0.,  3.,  5., 60.,  0.,  7.,  1.,  2.,  2.,  6.,  5.,  0.,  9.],
        [ 8.,  0.,  1.,  0., 10., 16.,  8., 37.,  0.,  9.,  1.,  7.,  1.,  1.,  1.],
        [ 6.,  1.,  1.,  0., 14.,  6.,  1., 49.,  0.,  3.,  1., 10.,  2.,  1.,  5.],
        [ 0.,  7.,  7.,  0.,  2.,  0.,  0.,  0., 70.,  0.,  8.,  0.,  3.,  3.,  0.],
        [ 6.,  1.,  0.,  0.,  3., 25.,  3., 31.,  1., 20.,  0.,  4.,  3.,  2.,  1.],
        [ 1., 22.,  7.,  1.,  2.,  0.,  0.,  0., 20.,  0., 44.,  0.,  1.,  2.,  0.],
        [ 2.,  0.,  4.,  0.,  9.,  6.,  1., 

Como puede observarse, la performance del modelo es bastante deficiente, dado que no llega a un accuracy del 0.6 en el conjunto de entrenamiento, siendo la performance en el conjunto de validación de un 0.46. La inspección visual de los resultados revela que el modelo es capaz de distinguir espacios abiertos de espacios cerrados, pero falla sistemáticamente a la hora de realizar diferenciaciones más finas entre dichos espacios. Por ejemplo, es común que el modelo confunda espacios como `Office` y `Bedroom`, `Living Room` o `Store`, pero es infrecuente encontrar fallos evidentes tales como confundir `Bedroom` con `Forest`. Sin embargo, estas equivocaciones siguen ocurriendo en algunos casos, lo que evidencia que el modelo no se encuentra optimizado. 

La aproximación evidente para solucionar este tipo de problema es realizar un aumento del vocabulario, lo que permitirá al modelo incrementar su aprendizaje de patrones sutiles en las imágenes. Empezaremos con un incremento drástico del tamaño de vocabulario (`500`) para ver si estamos incurriendo en *overfitting* del modelo.

## 1.2 Aumentar el tamaño del vocabulario

In [9]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=500)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [01:04<00:00, 46.27image/s]

Clustering descriptors into 500 words using K-means...


Adicionalmente, incrementaremos el número de iteraciones del modelo de SVM:

In [10]:
image_classifier = ImageClassifier(bow)
image_classifier.train(training_set, iterations=400)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [02:03<00:00, 24.23image/s]

Training SVM...


In [12]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             88.0    0.0     1.0      0.0         3.0          0.0      0.0         12.0       0.0     1.0           3.0    4.0     2.0     0.0            2.0
Coast                0.0  231.0     3.0      1.0         0.0          0.0      0.0          0.0      11.0     0.0          12.0    0.0     0.0     1.0            1.0
Forest               0.0    0.0   225.0      0.0         0.0          0.0      0.0          0.0       2.0     0.0           1.0    0.0     0.0     0.0            0.0
Highway              0.0    7.0     0.0    143.0         1.0          0.0      0.0          0.0       3.0     0.0           0.0    2.0     3.0     0.0            1.0
Industrial           0.0    2.0     0.0      0.0       168.0          8.0      1.0          2.0       3.0     1.0  

(0.8807370184254606,
 array([[ 88.,   0.,   1.,   0.,   3.,   0.,   0.,  12.,   0.,   1.,   3.,   4.,   2.,   0.,   2.],
        [  0., 231.,   3.,   1.,   0.,   0.,   0.,   0.,  11.,   0.,  12.,   0.,   0.,   1.,   1.],
        [  0.,   0., 225.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   1.,   0.,   0.,   0.,   0.],
        [  0.,   7.,   0., 143.,   1.,   0.,   0.,   0.,   3.,   0.,   0.,   2.,   3.,   0.,   1.],
        [  0.,   2.,   0.,   0., 168.,   8.,   1.,   2.,   3.,   1.,   0.,  13.,   7.,   3.,   3.],
        [  1.,   0.,   0.,   0.,   9., 178.,   0.,   6.,   0.,   0.,   1.,   5.,   4.,   0.,   4.],
        [  4.,   0.,   0.,   0.,   2.,   3.,  82.,  12.,   0.,   1.,   0.,   3.,   1.,   0.,   2.],
        [  3.,   0.,   0.,   0.,   6.,   2.,   3., 167.,   1.,   1.,   0.,   4.,   1.,   0.,   1.],
        [  0.,   4.,   6.,   0.,   0.,   0.,   0.,   0., 254.,   0.,  10.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   3.,   1.,   0.,  11.,   0., 100.,   0.,   0.

In [13]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             24.0    2.0     1.0      1.0         8.0          5.0      1.0         32.0       1.0     8.0           1.0    9.0     1.0     2.0            4.0
Coast                0.0   52.0     0.0     10.0         2.0          0.0      2.0          0.0      18.0     0.0          13.0    0.0     0.0     1.0            2.0
Forest               0.0    0.0    89.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0           6.0    0.0     0.0     0.0            0.0
Highway              0.0   17.0     0.0     47.0         9.0          4.0      3.0          0.0       3.0     1.0           7.0    0.0     4.0     1.0            4.0
Industrial           4.0    1.0     0.0      2.0        45.0          6.0      4.0          6.0       0.0     1.0

(0.5386666666666666,
 array([[24.,  2.,  1.,  1.,  8.,  5.,  1., 32.,  1.,  8.,  1.,  9.,  1.,  2.,  4.],
        [ 0., 52.,  0., 10.,  2.,  0.,  2.,  0., 18.,  0., 13.,  0.,  0.,  1.,  2.],
        [ 0.,  0., 89.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  6.,  0.,  0.,  0.,  0.],
        [ 0., 17.,  0., 47.,  9.,  4.,  3.,  0.,  3.,  1.,  7.,  0.,  4.,  1.,  4.],
        [ 4.,  1.,  0.,  2., 45.,  6.,  4.,  6.,  0.,  1.,  3., 11.,  8.,  3.,  6.],
        [ 2.,  0.,  0.,  0.,  9., 55.,  3.,  4.,  1.,  2.,  1.,  8.,  5.,  0., 10.],
        [ 5.,  0.,  0.,  1.,  3., 14., 19., 28.,  0., 11.,  1., 13.,  1.,  3.,  1.],
        [ 7.,  0.,  0.,  1.,  6.,  4.,  6., 53.,  1.,  6.,  0.,  8.,  4.,  0.,  4.],
        [ 0.,  8.,  6.,  1.,  0.,  0.,  0.,  0., 74.,  0.,  8.,  0.,  1.,  2.,  0.],
        [ 6.,  0.,  0.,  0.,  6.,  7., 11., 21.,  1., 40.,  0.,  7.,  1.,  0.,  0.],
        [ 0., 20.,  7.,  2.,  3.,  0.,  0.,  0., 15.,  0., 50.,  1.,  1.,  0.,  1.],
        [ 2.,  0.,  2.,  0.,  5.,  4.,  0., 

Podemos ver como con un vocabulario mucho más extenso hay un aumento significativo de la performance en el connjunto de training (88%). Sin embargo, la mejora de accuracy en el conjunto de validación es tan solo parcial, pasando de un 0,46 a un 0.56. Como era de esperar, el aumento del tamaño de vocabulario conlleva un incremento sustancial de overfitting a los datos, como se puede apreciar en la diferencia de performance entre el conjunto de validación y el conjunto de entrenamiento. Por otra parte, el coste computacional es a su vez mucho más elevado, puesto que el proceso de encontrar los clusters ha consumido una media hora, mientras que en el caso anterior pudimos obtener las features en menos de 10 minutos. 

## 1.3 Probar diferentes kernels para SVM

### RBF

A continuación procederemos a probar un kernel diferente para el clasificador, utilizando un tamaño intermedio de vocabulario (`250`)

In [21]:
# change SVM kernel to RBF and then reload
import image_classifier
reload(image_classifier)

<module 'image_classifier' from '/home/carlotamoh/Dropbox/Documents/MasterBD/ML2/Labs/ml2-deep-learning/BoW/image_classifier.py'>

In [22]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=250)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [01:00<00:00, 49.46image/s]

Clustering descriptors into 250 words using K-means...


En este caso volvemos a reducir el número de iteraciones del modelo SVM

In [23]:
image_classifier = ImageClassifier(bow)
image_classifier.train(training_set, iterations=100)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:48<00:00, 27.53image/s]

Training SVM...


In [24]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             35.0    4.0     2.0      0.0        13.0          5.0      1.0         22.0       5.0     2.0           4.0   12.0     5.0     2.0            4.0
Coast                0.0  205.0    10.0     10.0         0.0          2.0      0.0          1.0      16.0     1.0          11.0    0.0     1.0     1.0            2.0
Forest               0.0    0.0   211.0      0.0         1.0          0.0      0.0          0.0       6.0     0.0          10.0    0.0     0.0     0.0            0.0
Highway              0.0   27.0     0.0     86.0         6.0          3.0      0.0          0.0       9.0     1.0           4.0    2.0     8.0     3.0           11.0
Industrial           2.0    6.0     3.0      3.0       123.0         13.0      0.0          9.0      10.0     2.0  

(0.6737018425460637,
 array([[ 35.,   4.,   2.,   0.,  13.,   5.,   1.,  22.,   5.,   2.,   4.,  12.,   5.,   2.,   4.],
        [  0., 205.,  10.,  10.,   0.,   2.,   0.,   1.,  16.,   1.,  11.,   0.,   1.,   1.,   2.],
        [  0.,   0., 211.,   0.,   1.,   0.,   0.,   0.,   6.,   0.,  10.,   0.,   0.,   0.,   0.],
        [  0.,  27.,   0.,  86.,   6.,   3.,   0.,   0.,   9.,   1.,   4.,   2.,   8.,   3.,  11.],
        [  2.,   6.,   3.,   3., 123.,  13.,   0.,   9.,  10.,   2.,   2.,  10.,  13.,   4.,  11.],
        [  2.,   5.,   0.,   2.,  11., 148.,   0.,   5.,   0.,   1.,   2.,  12.,   9.,   1.,  10.],
        [  8.,   0.,   0.,   0.,  17.,  24.,  21.,  16.,   0.,  12.,   0.,   8.,   0.,   2.,   2.],
        [  7.,   1.,   0.,   0.,  15.,  10.,   3., 127.,   2.,   2.,   2.,  12.,   1.,   2.,   5.],
        [  0.,  16.,  17.,   0.,   0.,   0.,   0.,   0., 215.,   0.,  23.,   0.,   2.,   0.,   1.],
        [  3.,   0.,   0.,   0.,   8.,  12.,   2.,  21.,   0.,  62.,   0.,   3.

In [25]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             19.0    4.0     2.0      1.0         9.0          6.0      2.0         30.0       1.0     6.0           3.0    8.0     3.0     1.0            5.0
Coast                1.0   65.0     0.0      8.0         1.0          0.0      0.0          1.0      11.0     0.0          11.0    0.0     0.0     1.0            1.0
Forest               0.0    0.0    85.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0          10.0    0.0     0.0     0.0            0.0
Highway              0.0   29.0     0.0     36.0         9.0          7.0      1.0          0.0       5.0     0.0           2.0    1.0     4.0     1.0            5.0
Industrial           3.0    4.0     0.0      0.0        29.0         15.0      1.0          8.0       4.0     1.0

(0.49666666666666665,
 array([[19.,  4.,  2.,  1.,  9.,  6.,  2., 30.,  1.,  6.,  3.,  8.,  3.,  1.,  5.],
        [ 1., 65.,  0.,  8.,  1.,  0.,  0.,  1., 11.,  0., 11.,  0.,  0.,  1.,  1.],
        [ 0.,  0., 85.,  0.,  0.,  0.,  0.,  0.,  5.,  0., 10.,  0.,  0.,  0.,  0.],
        [ 0., 29.,  0., 36.,  9.,  7.,  1.,  0.,  5.,  0.,  2.,  1.,  4.,  1.,  5.],
        [ 3.,  4.,  0.,  0., 29., 15.,  1.,  8.,  4.,  1.,  3., 10., 15.,  0.,  7.],
        [ 0.,  1.,  0.,  2.,  7., 59.,  1.,  4.,  2.,  3.,  0.,  4.,  6.,  0., 11.],
        [ 9.,  0.,  0.,  1.,  6., 18., 14., 29.,  0., 11.,  1.,  6.,  2.,  1.,  2.],
        [ 7.,  1.,  0.,  0., 14.,  5.,  6., 43.,  1.,  7.,  1.,  7.,  4.,  0.,  4.],
        [ 0.,  9.,  5.,  0.,  0.,  0.,  0.,  0., 69.,  0., 13.,  0.,  4.,  0.,  0.],
        [ 5.,  0.,  0.,  0., 10., 18.,  1., 27.,  1., 32.,  0.,  3.,  1.,  1.,  1.],
        [ 0., 29.,  5.,  1.,  2.,  0.,  0.,  0., 19.,  0., 40.,  1.,  2.,  1.,  0.],
        [ 1.,  0.,  4.,  0., 13.,  7.,  0.,

Como podemos observar, hay un ligero aumento de la performance en el dataset de training, pero no se observa un cambio significativo en el conjunto de validación. Iteremos una vez más con otro tipo de kernel, esta vez basado en la intersección de histogramas

### Histogram based kernel

In [26]:
# change back kernel and reload
import image_classifier
reload(image_classifier)

<module 'image_classifier' from '/home/carlotamoh/Dropbox/Documents/MasterBD/ML2/Labs/ml2-deep-learning/BoW/image_classifier.py'>

In [27]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=250)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [00:57<00:00, 51.48image/s]

Clustering descriptors into 250 words using K-means...


En este caso volvemos a reducir el número de iteraciones del modelo SVM

In [28]:
image_classifier = ImageClassifier(bow)
image_classifier.train(training_set, iterations=100)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:45<00:00, 28.19image/s]

Training SVM...


In [29]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             88.0    0.0     2.0      0.0         4.0          1.0      2.0         11.0       0.0     2.0           0.0    2.0     1.0     2.0            1.0
Coast                0.0  142.0     4.0     26.0         1.0          0.0      0.0          0.0      17.0     0.0          66.0    0.0     0.0     3.0            1.0
Forest               0.0    0.0   225.0      0.0         0.0          0.0      0.0          0.0       2.0     0.0           1.0    0.0     0.0     0.0            0.0
Highway              0.0   14.0     0.0    136.0         1.0          0.0      0.0          0.0       0.0     0.0           1.0    2.0     1.0     0.0            5.0
Industrial           3.0    0.0     1.0      5.0       152.0          9.0      1.0          3.0       1.0     3.0  

(0.7624790619765495,
 array([[ 88.,   0.,   2.,   0.,   4.,   1.,   2.,  11.,   0.,   2.,   0.,   2.,   1.,   2.,   1.],
        [  0., 142.,   4.,  26.,   1.,   0.,   0.,   0.,  17.,   0.,  66.,   0.,   0.,   3.,   1.],
        [  0.,   0., 225.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   1.,   0.,   0.,   0.,   0.],
        [  0.,  14.,   0., 136.,   1.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,   1.,   0.,   5.],
        [  3.,   0.,   1.,   5., 152.,   9.,   1.,   3.,   1.,   3.,   3.,  15.,   7.,   5.,   3.],
        [  1.,   0.,   0.,   3.,  20., 158.,   0.,   5.,   0.,   0.,   0.,   4.,   4.,   1.,  12.],
        [  2.,   0.,   0.,   0.,   8.,   7.,  76.,   9.,   0.,   1.,   0.,   5.,   0.,   1.,   1.],
        [ 16.,   0.,   0.,   0.,   6.,   2.,  11., 143.,   1.,   2.,   0.,   6.,   1.,   0.,   1.],
        [  0.,  12.,  14.,   1.,   1.,   0.,   0.,   0., 223.,   0.,  20.,   0.,   1.,   1.,   1.],
        [  0.,   0.,   0.,   0.,   3.,   1.,   3.,   6.,   0., 101.,   0.,   0.

In [30]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             28.0    2.0     0.0      4.0         6.0          4.0      4.0         28.0       1.0     5.0           2.0   11.0     2.0     1.0            2.0
Coast                3.0   50.0     1.0     15.0         2.0          1.0      0.0          0.0      12.0     0.0          15.0    0.0     0.0     0.0            1.0
Forest               0.0    0.0    90.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0           4.0    0.0     1.0     0.0            0.0
Highway              1.0   17.0     0.0     50.0        10.0          6.0      1.0          0.0       4.0     0.0           4.0    0.0     2.0     1.0            4.0
Industrial           7.0    1.0     0.0      2.0        39.0         13.0      3.0          6.0       1.0     1.0

(0.5093333333333333,
 array([[28.,  2.,  0.,  4.,  6.,  4.,  4., 28.,  1.,  5.,  2., 11.,  2.,  1.,  2.],
        [ 3., 50.,  1., 15.,  2.,  1.,  0.,  0., 12.,  0., 15.,  0.,  0.,  0.,  1.],
        [ 0.,  0., 90.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,  4.,  0.,  1.,  0.,  0.],
        [ 1., 17.,  0., 50., 10.,  6.,  1.,  0.,  4.,  0.,  4.,  0.,  2.,  1.,  4.],
        [ 7.,  1.,  0.,  2., 39., 13.,  3.,  6.,  1.,  1.,  2., 11.,  9.,  2.,  3.],
        [ 4.,  0.,  0.,  3., 11., 55.,  4.,  6.,  0.,  1.,  1.,  6.,  4.,  1.,  4.],
        [14.,  0.,  0.,  1.,  8., 15., 18., 20.,  0., 15.,  1.,  6.,  0.,  1.,  1.],
        [12.,  1.,  0.,  0., 10.,  4.,  7., 44.,  1.,  9.,  0.,  4.,  4.,  1.,  3.],
        [ 1.,  7.,  8.,  1.,  0.,  0.,  0.,  0., 71.,  0.,  7.,  0.,  3.,  2.,  0.],
        [ 5.,  0.,  0.,  1.,  6.,  8., 14., 17.,  0., 46.,  0.,  2.,  0.,  0.,  1.],
        [ 1., 16.,  8.,  3.,  2.,  0.,  0.,  0., 20.,  0., 43.,  1.,  3.,  2.,  1.],
        [ 2.,  0.,  3.,  0.,  9., 10.,  1., 

De forma similar a lo observado para el kernel anterior, podemos observar un aumento en la performance del modelo en el conjunto de train, pero este no se ve acompañado por una mejora sustancial de la performance en validation.

## 1.4 Utilizar un matcher diferente

Otra de las opciones a testar es el tipo de `matcher` que empleamos a la hora de realizar la comparación de las features extraídas en la imagen. Por defecto, la clase `image_classifier` emplea `FLANN` como feature matcher, pero podemos comparar los resultados con el uso de otro matcher (en este caso, `Brute-Force`).

Como no hemos observado diferencias significativas en la performance del modelo variando el kernel de SVM, nos decantaremos por mantener el kernel lineal. En este caso trataremos de aumentar ligeramente el vocabulario, pasando de `250` a `300`. 

In [2]:
training_set = Dataset.load('../dataset/dataset/training', '*.jpg')
validation_set = Dataset.load('../dataset/dataset/validation', '*.jpg')

In [3]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=300)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [00:58<00:00, 50.70image/s]

Clustering descriptors into 300 words using K-means...


Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [4]:
# get vocabulary out
vocabulary = bow.vocabulary
vocabulary

array([[16.760193 , 15.775318 , 20.73082  , ..., 24.417799 , 78.82179  ,
        71.250984 ],
       [68.93086  , 68.35389  , 22.78401  , ..., 16.368431 , 12.79426  ,
        12.925643 ],
       [23.91897  ,  6.081416 ,  3.4714396, ..., 12.829284 ,  9.203799 ,
         9.845308 ],
       ...,
       [15.921015 , 18.223238 , 23.526295 , ..., 33.397423 , 42.511745 ,
        25.688423 ],
       [76.65834  , 12.746878 ,  7.470962 , ..., 11.224414 ,  9.904449 ,
        16.421871 ],
       [16.363153 , 13.420173 , 15.246833 , ...,  8.418905 , 10.0714655,
        13.986823 ]], dtype=float32)

In [5]:
image_classifier = ImageClassifier(bow)
# initialize Brute-Force as matcher
image_classifier._initialize_feature_matcher('Brute-Force')
image_classifier.train(training_set, iterations=50)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:45<00:00, 28.28image/s]

Training SVM...


In [6]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom            110.0    0.0     0.0      0.0         0.0          0.0      0.0          5.0       0.0     0.0           0.0    0.0     1.0     0.0            0.0
Coast                0.0  215.0     0.0     11.0         0.0          0.0      0.0          0.0       8.0     0.0          26.0    0.0     0.0     0.0            0.0
Forest               0.0    0.0   228.0      0.0         0.0          0.0      0.0          0.0       0.0     0.0           0.0    0.0     0.0     0.0            0.0
Highway              0.0    1.0     0.0    159.0         0.0          0.0      0.0          0.0       0.0     0.0           0.0    0.0     0.0     0.0            0.0
Industrial           4.0    0.0     0.0      1.0       172.0          3.0      2.0          4.0       0.0     1.0  

(0.8820770519262982,
 array([[110.,   0.,   0.,   0.,   0.,   0.,   0.,   5.,   0.,   0.,   0.,   0.,   1.,   0.,   0.],
        [  0., 215.,   0.,  11.,   0.,   0.,   0.,   0.,   8.,   0.,  26.,   0.,   0.,   0.,   0.],
        [  0.,   0., 228.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   1.,   0., 159.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  4.,   0.,   0.,   1., 172.,   3.,   2.,   4.,   0.,   1.,   0.,   8.,  12.,   0.,   4.],
        [  1.,   0.,   0.,   0.,  13., 185.,   0.,   0.,   0.,   0.,   0.,   4.,   1.,   0.,   4.],
        [  3.,   0.,   0.,   0.,   0.,   0., 106.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  4.,   0.,   0.,   0.,   7.,   0.,  10., 161.,   0.,   4.,   0.,   2.,   0.,   1.,   0.],
        [  1.,  10.,   6.,   0.,   2.,   0.,   0.,   0., 230.,   0.,  25.,   0.,   0.,   0.,   0.],
        [  1.,   0.,   0.,   0.,   1.,   1.,   0.,   2.,   0., 110.,   0.,   0.

In [8]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             28.0    3.0     0.0      3.0         8.0          4.0      4.0         29.0       3.0     6.0           0.0    6.0     2.0     1.0            3.0
Coast                2.0   52.0     0.0     12.0         0.0          0.0      0.0          0.0      14.0     0.0          19.0    0.0     0.0     0.0            1.0
Forest               0.0    0.0    84.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0          11.0    0.0     0.0     0.0            0.0
Highway              0.0   21.0     0.0     59.0         2.0          4.0      0.0          0.0       1.0     0.0           8.0    0.0     2.0     0.0            3.0
Industrial           9.0    3.0     0.0      5.0        30.0         12.0      6.0          1.0       1.0     6.0

(0.498,
 array([[28.,  3.,  0.,  3.,  8.,  4.,  4., 29.,  3.,  6.,  0.,  6.,  2.,  1.,  3.],
        [ 2., 52.,  0., 12.,  0.,  0.,  0.,  0., 14.,  0., 19.,  0.,  0.,  0.,  1.],
        [ 0.,  0., 84.,  0.,  0.,  0.,  0.,  0.,  5.,  0., 11.,  0.,  0.,  0.,  0.],
        [ 0., 21.,  0., 59.,  2.,  4.,  0.,  0.,  1.,  0.,  8.,  0.,  2.,  0.,  3.],
        [ 9.,  3.,  0.,  5., 30., 12.,  6.,  1.,  1.,  6.,  1.,  8.,  9.,  3.,  6.],
        [ 4.,  0.,  0.,  3., 12., 47.,  6.,  4.,  0.,  2.,  0.,  8.,  3.,  1., 10.],
        [15.,  0.,  0.,  1.,  6., 11., 29., 22.,  0., 11.,  0.,  3.,  2.,  0.,  0.],
        [ 6.,  0.,  0.,  1., 16.,  5.,  9., 37.,  0., 14.,  0.,  3.,  2.,  1.,  6.],
        [ 1., 13.,  8.,  0.,  0.,  0.,  0.,  0., 58.,  0., 19.,  0.,  0.,  1.,  0.],
        [ 8.,  0.,  0.,  2.,  5.,  9., 17., 16.,  1., 37.,  0.,  2.,  2.,  0.,  1.],
        [ 2., 16.,  8.,  6.,  1.,  0.,  0.,  0., 21.,  0., 40.,  0.,  3.,  1.,  2.],
        [ 0.,  0.,  1.,  0.,  9.,  8.,  4.,  6.,  0.,  2.

Como podemos observar, aunque hay un incremento en la performance en el conjunto de train, esto no se ve reflejado en un aumento en la performance en el conjunto de test. Además, es posible que este aumento se deba no al cambio de matcher, sino al cambio en el tamaño del vocabulario.

## 1.5 Cambios en el algoritmo de Feature Extractor

Otro de los posibles cambios a implementar es utilizar un algoritmo diferente para la extracción de features. Existen muchas opciones a considerar para realizar esto, pero en este caso exploraremos dos: KAZE y FREAK.

En el caso de FREAK, usaremos este algoritmo únicamente como extractor de los descriptores, mientras que los keypoints serán extraídos con FAST.

También tratamos de explorar el uso de SURF, un algoritmo bastante popular, pero en la actualidad se encuentra patentado, por lo que la librería opencv ha dejado de incluirlo.

### FAST + FREAK

In [2]:
training_set = Dataset.load('../dataset/dataset/training', '*.jpg')
validation_set = Dataset.load('../dataset/dataset/validation', '*.jpg')

Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [3]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='FREAK', vocabulary_size=250)


BUILDING DICTIONARY
100%|██████████| 2985/2985 [00:25<00:00, 118.08image/s]

Clustering descriptors into 250 words using K-means...


Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [ ]:
# get vocabulary out
vocabulary = bow.vocabulary
vocabulary

array([[16.760193 , 15.775318 , 20.73082  , ..., 24.417799 , 78.82179  ,
        71.250984 ],
       [68.93086  , 68.35389  , 22.78401  , ..., 16.368431 , 12.79426  ,
        12.925643 ],
       [23.91897  ,  6.081416 ,  3.4714396, ..., 12.829284 ,  9.203799 ,
         9.845308 ],
       ...,
       [15.921015 , 18.223238 , 23.526295 , ..., 33.397423 , 42.511745 ,
        25.688423 ],
       [76.65834  , 12.746878 ,  7.470962 , ..., 11.224414 ,  9.904449 ,
        16.421871 ],
       [16.363153 , 13.420173 , 15.246833 , ...,  8.418905 , 10.0714655,
        13.986823 ]], dtype=float32)

In [ ]:
image_classifier = ImageClassifier(bow)
# initialize Brute-Force as matcher
image_classifier.train(training_set, iterations=50)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [01:45<00:00, 28.28image/s]

Training SVM...


In [ ]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom            110.0    0.0     0.0      0.0         0.0          0.0      0.0          5.0       0.0     0.0           0.0    0.0     1.0     0.0            0.0
Coast                0.0  215.0     0.0     11.0         0.0          0.0      0.0          0.0       8.0     0.0          26.0    0.0     0.0     0.0            0.0
Forest               0.0    0.0   228.0      0.0         0.0          0.0      0.0          0.0       0.0     0.0           0.0    0.0     0.0     0.0            0.0
Highway              0.0    1.0     0.0    159.0         0.0          0.0      0.0          0.0       0.0     0.0           0.0    0.0     0.0     0.0            0.0
Industrial           4.0    0.0     0.0      1.0       172.0          3.0      2.0          4.0       0.0     1.0  

(0.8820770519262982,
 array([[110.,   0.,   0.,   0.,   0.,   0.,   0.,   5.,   0.,   0.,   0.,   0.,   1.,   0.,   0.],
        [  0., 215.,   0.,  11.,   0.,   0.,   0.,   0.,   8.,   0.,  26.,   0.,   0.,   0.,   0.],
        [  0.,   0., 228.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   1.,   0., 159.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  4.,   0.,   0.,   1., 172.,   3.,   2.,   4.,   0.,   1.,   0.,   8.,  12.,   0.,   4.],
        [  1.,   0.,   0.,   0.,  13., 185.,   0.,   0.,   0.,   0.,   0.,   4.,   1.,   0.,   4.],
        [  3.,   0.,   0.,   0.,   0.,   0., 106.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  4.,   0.,   0.,   0.,   7.,   0.,  10., 161.,   0.,   4.,   0.,   2.,   0.,   1.,   0.],
        [  1.,  10.,   6.,   0.,   2.,   0.,   0.,   0., 230.,   0.,  25.,   0.,   0.,   0.,   0.],
        [  1.,   0.,   0.,   0.,   1.,   1.,   0.,   2.,   0., 110.,   0.,   0.

In [ ]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             28.0    3.0     0.0      3.0         8.0          4.0      4.0         29.0       3.0     6.0           0.0    6.0     2.0     1.0            3.0
Coast                2.0   52.0     0.0     12.0         0.0          0.0      0.0          0.0      14.0     0.0          19.0    0.0     0.0     0.0            1.0
Forest               0.0    0.0    84.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0          11.0    0.0     0.0     0.0            0.0
Highway              0.0   21.0     0.0     59.0         2.0          4.0      0.0          0.0       1.0     0.0           8.0    0.0     2.0     0.0            3.0
Industrial           9.0    3.0     0.0      5.0        30.0         12.0      6.0          1.0       1.0     6.0

(0.498,
 array([[28.,  3.,  0.,  3.,  8.,  4.,  4., 29.,  3.,  6.,  0.,  6.,  2.,  1.,  3.],
        [ 2., 52.,  0., 12.,  0.,  0.,  0.,  0., 14.,  0., 19.,  0.,  0.,  0.,  1.],
        [ 0.,  0., 84.,  0.,  0.,  0.,  0.,  0.,  5.,  0., 11.,  0.,  0.,  0.,  0.],
        [ 0., 21.,  0., 59.,  2.,  4.,  0.,  0.,  1.,  0.,  8.,  0.,  2.,  0.,  3.],
        [ 9.,  3.,  0.,  5., 30., 12.,  6.,  1.,  1.,  6.,  1.,  8.,  9.,  3.,  6.],
        [ 4.,  0.,  0.,  3., 12., 47.,  6.,  4.,  0.,  2.,  0.,  8.,  3.,  1., 10.],
        [15.,  0.,  0.,  1.,  6., 11., 29., 22.,  0., 11.,  0.,  3.,  2.,  0.,  0.],
        [ 6.,  0.,  0.,  1., 16.,  5.,  9., 37.,  0., 14.,  0.,  3.,  2.,  1.,  6.],
        [ 1., 13.,  8.,  0.,  0.,  0.,  0.,  0., 58.,  0., 19.,  0.,  0.,  1.,  0.],
        [ 8.,  0.,  0.,  2.,  5.,  9., 17., 16.,  1., 37.,  0.,  2.,  2.,  0.,  1.],
        [ 2., 16.,  8.,  6.,  1.,  0.,  0.,  0., 21.,  0., 40.,  0.,  3.,  1.,  2.],
        [ 0.,  0.,  1.,  0.,  9.,  8.,  4.,  6.,  0.,  2.

Con la aproximación de FAST + FREAK no vemos diferencias en la performance con respecto a las iteraciones anteriores.

### KAZE

In [4]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='KAZE', vocabulary_size=250)


BUILDING DICTIONARY

Computing KAZE descriptors...
100%|██████████| 2985/2985 [02:43<00:00, 18.23image/s]

Clustering descriptors into 250 words using K-means...


Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [5]:
# get vocabulary out
vocabulary = bow.vocabulary
vocabulary

array([[-0.03454349,  0.01370027,  0.10040715, ...,  0.00303901,
         0.0701693 ,  0.06725428],
       [ 0.00452014,  0.00721528,  0.07825565, ..., -0.0106509 ,
         0.08514273,  0.09481083],
       [ 0.0698275 ,  0.02313369,  0.18186459, ...,  0.00524329,
         0.04784739,  0.05912996],
       ...,
       [-0.00183064, -0.00178427,  0.02830743, ..., -0.21246006,
         0.07682153,  0.2610173 ],
       [-0.0096981 , -0.00175046,  0.05642617, ...,  0.04005411,
         0.07017621,  0.09973915],
       [-0.00186627, -0.07073623,  0.04120081, ..., -0.0127933 ,
         0.03726632,  0.06509835]], dtype=float32)

In [6]:
image_classifier = ImageClassifier(bow)
# initialize Brute-Force as matcher
image_classifier.train(training_set, iterations=50)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [08:07<00:00,  6.12image/s]  

Training SVM...


In [7]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             73.0    6.0     3.0      0.0         2.0          5.0      5.0          5.0       1.0     4.0           3.0    1.0     1.0     1.0            6.0
Coast                1.0  185.0     6.0      4.0         3.0          0.0      2.0          0.0      11.0     2.0          40.0    0.0     1.0     0.0            5.0
Forest               1.0    2.0   194.0      0.0         0.0          1.0      0.0          0.0       8.0     1.0          15.0    3.0     2.0     0.0            1.0
Highway              0.0   44.0     1.0     79.0         3.0          3.0      1.0          1.0       9.0     1.0          10.0    0.0     2.0     1.0            5.0
Industrial           3.0   17.0    10.0      4.0        84.0          6.0      4.0          6.0      14.0     1.0  

(0.5855946398659967,
 array([[ 73.,   6.,   3.,   0.,   2.,   5.,   5.,   5.,   1.,   4.,   3.,   1.,   1.,   1.,   6.],
        [  1., 185.,   6.,   4.,   3.,   0.,   2.,   0.,  11.,   2.,  40.,   0.,   1.,   0.,   5.],
        [  1.,   2., 194.,   0.,   0.,   1.,   0.,   0.,   8.,   1.,  15.,   3.,   2.,   0.,   1.],
        [  0.,  44.,   1.,  79.,   3.,   3.,   1.,   1.,   9.,   1.,  10.,   0.,   2.,   1.,   5.],
        [  3.,  17.,  10.,   4.,  84.,   6.,   4.,   6.,  14.,   1.,  12.,  15.,  17.,   4.,  14.],
        [  6.,  12.,   5.,   0.,   7., 115.,   7.,   5.,   1.,   2.,   3.,  14.,   4.,   5.,  22.],
        [  9.,   5.,   0.,   2.,   5.,  12.,  51.,   5.,   0.,   6.,   2.,   2.,   2.,   3.,   6.],
        [ 31.,   4.,   5.,   1.,  12.,   9.,  17.,  88.,   2.,   7.,   0.,   5.,   0.,   1.,   7.],
        [  1.,  29.,  23.,   1.,   3.,   0.,   0.,   0., 168.,   0.,  38.,   2.,   6.,   0.,   3.],
        [  6.,   7.,   0.,   0.,   3.,   6.,   5.,   8.,   3.,  64.,   5.,   1.

In [8]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom             23.0    9.0     5.0      1.0         4.0         10.0      7.0         24.0       5.0     4.0           0.0    3.0     1.0     1.0            3.0
Coast                1.0   50.0     4.0     11.0         1.0          0.0      1.0          1.0      10.0     0.0          15.0    0.0     1.0     0.0            3.0
Forest               0.0    2.0    70.0      0.0         1.0          0.0      0.0          0.0       6.0     0.0          11.0    7.0     1.0     0.0            2.0
Highway              0.0   44.0     0.0     35.0         2.0          3.0      2.0          1.0       3.0     1.0           9.0    0.0     0.0     0.0            0.0
Industrial           7.0   13.0     4.0      1.0        26.0         10.0      2.0          2.0       8.0     0.0

(0.37383177570093457,
 array([[23.,  9.,  5.,  1.,  4., 10.,  7., 24.,  5.,  4.,  0.,  3.,  1.,  1.,  3.],
        [ 1., 50.,  4., 11.,  1.,  0.,  1.,  1., 10.,  0., 15.,  0.,  1.,  0.,  3.],
        [ 0.,  2., 70.,  0.,  1.,  0.,  0.,  0.,  6.,  0., 11.,  7.,  1.,  0.,  2.],
        [ 0., 44.,  0., 35.,  2.,  3.,  2.,  1.,  3.,  1.,  9.,  0.,  0.,  0.,  0.],
        [ 7., 13.,  4.,  1., 26., 10.,  2.,  2.,  8.,  0.,  6.,  7.,  8.,  1.,  5.],
        [ 3.,  6.,  2.,  0.,  4., 39.,  9.,  2.,  2.,  2.,  2.,  8.,  4.,  1., 16.],
        [16.,  3.,  2.,  0.,  2., 17., 20., 12.,  0.,  9.,  1.,  2.,  3.,  3., 10.],
        [19.,  4.,  0.,  1.,  4.,  3.,  6., 35.,  3.,  7.,  0.,  5.,  1.,  5.,  7.],
        [ 0., 20., 11.,  0.,  1.,  0.,  0.,  0., 47.,  0.,  8.,  4.,  4.,  1.,  4.],
        [ 9.,  6.,  0.,  1.,  1., 20.,  5., 13.,  2., 30.,  2.,  2.,  0.,  2.,  7.],
        [ 1., 26.,  8.,  4.,  5.,  0.,  1.,  0., 15.,  0., 33.,  1.,  3.,  1.,  2.],
        [ 2.,  2.,  9.,  0.,  4.,  9.,  3.,

Este modelo obtiene el peor rendimiento hasta el momento. De estas pruebas concluimos que no hay mejora alguna en el cambio de Feature Extractor

# 2. Modelo Random Forest

Una vez hemos explorado de manera amplia el abanico de posibilidades ofrecido por diferentes combinaciones de parámetros, procedemos a cambiar el modelo empleado para realizar la clasificación de las imágenes, empleando `AdaBoost`.

In [5]:
training_set = Dataset.load('../dataset/dataset/training', '*.jpg')
validation_set = Dataset.load('../dataset/dataset/validation', '*.jpg')

In [6]:
# instantiate class
bow = BoW()
# build vocabulary
bow.build_vocabulary(training_set=training_set, feature_type='SIFT', vocabulary_size=250)


BUILDING DICTIONARY

Computing SIFT descriptors...
100%|██████████| 2985/2985 [00:59<00:00, 49.93image/s]

Clustering descriptors into 250 words using K-means...


Una vez hemos realizado el clustering de decriptores podemos acceder al vocabulario guardándolo en una variable:

In [7]:
# get vocabulary out
vocabulary = bow.vocabulary
vocabulary

array([[28.74362  , 26.253588 , 24.66208  , ...,  5.7635565,  7.6453347,
        10.29984  ],
       [22.31875  ,  8.2664585,  9.425834 , ..., 22.165    ,  6.9550004,
         8.234167 ],
       [13.652828 , 11.948774 , 18.131586 , ..., 10.556203 , 15.390143 ,
        21.468075 ],
       ...,
       [29.49408  , 30.743656 , 18.806803 , ..., 12.71246  ,  9.512874 ,
        12.807931 ],
       [38.459354 , 31.205067 , 13.029983 , ..., 12.989578 ,  8.284753 ,
        11.781145 ],
       [40.784748 , 15.512742 , 10.736129 , ...,  8.017893 ,  8.382794 ,
        15.116212 ]], dtype=float32)

In [8]:
image_classifier = ImageClassifierRF(bow)
image_classifier.train(training_set)



TRAINING CLASSIFIER

Extracting features...
100%|██████████| 2985/2985 [03:49<00:00, 13.01image/s]

Training Random Forest...


Una vez hemos entrenado el modelo podemos analizar su performance en el conjunto de entrenamiento:

In [9]:
image_classifier.predict(training_set, 'Training', save=False)



CLASSIFICATION RESULTS (TRAINING)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom              0.0    3.0     0.0      0.0         0.0          6.0      0.0         62.0      16.0     0.0           7.0   19.0     0.0     0.0            3.0
Coast                0.0  210.0     3.0      0.0         1.0          0.0      0.0          0.0      22.0     0.0          23.0    0.0     0.0     1.0            0.0
Forest               0.0    0.0   210.0      0.0         0.0          0.0      0.0          0.0       5.0     0.0           8.0    5.0     0.0     0.0            0.0
Highway              0.0   64.0     0.0     13.0         0.0          1.0      0.0          5.0      32.0     0.0          28.0    5.0     0.0     3.0            9.0
Industrial           0.0    8.0     1.0      0.0         3.0         20.0      0.0         43.0      31.0     0.0  

(0.5373534338358459,
 array([[  0.,   3.,   0.,   0.,   0.,   6.,   0.,  62.,  16.,   0.,   7.,  19.,   0.,   0.,   3.],
        [  0., 210.,   3.,   0.,   1.,   0.,   0.,   0.,  22.,   0.,  23.,   0.,   0.,   1.,   0.],
        [  0.,   0., 210.,   0.,   0.,   0.,   0.,   0.,   5.,   0.,   8.,   5.,   0.,   0.,   0.],
        [  0.,  64.,   0.,  13.,   0.,   1.,   0.,   5.,  32.,   0.,  28.,   5.,   0.,   3.,   9.],
        [  0.,   8.,   1.,   0.,   3.,  20.,   0.,  43.,  31.,   0.,   7.,  84.,   0.,   6.,   8.],
        [  0.,   2.,   1.,   0.,   0., 134.,   0.,  24.,   0.,   0.,   1.,  34.,   0.,   1.,  11.],
        [  0.,   2.,   0.,   0.,   0.,  20.,   0.,  70.,   1.,   0.,   1.,  15.,   0.,   0.,   1.],
        [  0.,   0.,   1.,   0.,   1.,   5.,   0., 143.,   3.,   0.,   1.,  29.,   0.,   4.,   2.],
        [  0.,  23.,  14.,   0.,   0.,   0.,   0.,   0., 205.,   0.,  31.,   1.,   0.,   0.,   0.],
        [  0.,   1.,   0.,   0.,   0.,  10.,   0.,  87.,   3.,   0.,   0.,  11.

Procedemos a continuación a evaluar el modelo en el conjunto de test

In [10]:
image_classifier.predict(validation_set, 'Validation', save=False)



CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  Bedroom  Coast  Forest  Highway  Industrial  Inside city  Kitchen  Living room  Mountain  Office  Open country  Store  Street  Suburb  Tall building
Bedroom              0.0    8.0     1.0      0.0         0.0          6.0      0.0         50.0       8.0     0.0           8.0   16.0     0.0     0.0            3.0
Coast                0.0   72.0     0.0      0.0         0.0          0.0      0.0          0.0      12.0     0.0          16.0    0.0     0.0     0.0            0.0
Forest               0.0    0.0    76.0      0.0         0.0          0.0      0.0          0.0       7.0     0.0           9.0    8.0     0.0     0.0            0.0
Highway              0.0   55.0     0.0      1.0         0.0          5.0      0.0          5.0      13.0     0.0          14.0    4.0     0.0     3.0            0.0
Industrial           0.0    6.0     0.0      0.0         1.0          7.0      0.0         19.0      14.0     0.0

(0.36133333333333334,
 array([[ 0.,  8.,  1.,  0.,  0.,  6.,  0., 50.,  8.,  0.,  8., 16.,  0.,  0.,  3.],
        [ 0., 72.,  0.,  0.,  0.,  0.,  0.,  0., 12.,  0., 16.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 76.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  9.,  8.,  0.,  0.,  0.],
        [ 0., 55.,  0.,  1.,  0.,  5.,  0.,  5., 13.,  0., 14.,  4.,  0.,  3.,  0.],
        [ 0.,  6.,  0.,  0.,  1.,  7.,  0., 19., 14.,  0.,  5., 44.,  0.,  0.,  4.],
        [ 0.,  2.,  1.,  0.,  0., 44.,  0., 15.,  1.,  0.,  2., 26.,  2.,  0.,  7.],
        [ 0.,  0.,  0.,  0.,  0., 16.,  0., 60.,  2.,  0.,  0., 18.,  0.,  1.,  3.],
        [ 0.,  1.,  0.,  0.,  0.,  3.,  0., 68.,  5.,  0.,  0., 18.,  0.,  2.,  3.],
        [ 0.,  8.,  8.,  0.,  0.,  0.,  0.,  0., 60.,  0., 22.,  1.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., 18.,  0., 68.,  2.,  0.,  1., 10.,  0.,  0.,  0.],
        [ 0., 18.,  5.,  0.,  0.,  0.,  0.,  0., 21.,  0., 55.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  7.,  0.,

Como podemos observar, la performance del nuevo modelo es mucho peor a la que obteníamos en el caso de SVM, tanto para el dataset de training como el de validation.